# artsy

In [1]:
from pymongo import MongoClient
import requests

import credentials

## setup

### database

In [2]:
client = MongoClient()
# client.drop_database('artsy')
db = client.artsy

### base url

In [3]:
BASE_URL = 'https://api.artsy.net/api'

### app

In [4]:
app_name = credentials.artsy_app_name
client_id = credentials.artsy_client_id
client_secret = credentials.artsy_client_secret

### token

In [5]:
headers = {}
payload = {'client_id': client_id, 'client_secret': client_secret}
url = BASE_URL + '/tokens/xapp_token'
response = requests.post(url, params=payload, headers=headers)
token = response.json()['token']

### auxiliar

In [6]:
def change_id_field(d):
    d['_id'] = d.pop('id')

## artsy resources
* Status
* Artists
* Artworks
* Editions
* Genes
* Profiles
* Partners
* Shows
* Fairs
* Applications
* Users
* User Details
* Collections
* Collection Items
* Images
* Search

## status

In [8]:
headers = {}
payload = {}
url = BASE_URL + '/status'
response = requests.get(url, params=payload, headers=headers)
status = response.json()['status']
print('Status: %s' % status)

Status: good


## resources

### artists, artworks, genes, profiles, partners, shows, fairs

In [9]:
headers = {'X-Xapp-Token': token}
resources = ['artists', 'artworks', 'genes', 'profiles', 'partners', 'shows', 'fairs']
for resource in resources:
    print()
    print('Processing %s...' % resource)
    url = BASE_URL + '/' + resource
    
    # total count
    payload = {'total_count': 1}
    response = requests.get(url, params=payload, headers=headers)
    total_count = response.json()['total_count']
    print('Total %s: %d' % (resource, total_count))
    
    # data
    payload = {}
    count = 0
    while True:
        response = requests.get(url, params=payload, headers=headers)
        data = response.json()['_embedded'][resource]
        if not data:
            break
        [change_id_field(d) for d in data]
        db[resource].insert_many(data)
        url = response.json()['_links']['next']['href']
        if not url:
            break
        count += len(data)
        if count % (total_count / 10) < len(data):
            print('%d %s processed.' % (count, resource))


Processing artists...
Total artists: 49934
4995 artists processed.
9990 artists processed.
14985 artists processed.
19975 artists processed.
24970 artists processed.
29965 artists processed.
34955 artists processed.
39950 artists processed.
44945 artists processed.
49934 artists processed.

Processing artworks...
Total artworks: 26074
2610 artworks processed.
5215 artworks processed.
7825 artworks processed.
10430 artworks processed.
13040 artworks processed.
15645 artworks processed.
18255 artworks processed.
20860 artworks processed.
23470 artworks processed.

Processing genes...
Total genes: 921
95 genes processed.
185 genes processed.
280 genes processed.
370 genes processed.
465 genes processed.
555 genes processed.
645 genes processed.
740 genes processed.
830 genes processed.
921 genes processed.

Processing profiles...
Total profiles: 2466
250 profiles processed.
495 profiles processed.
740 profiles processed.
990 profiles processed.
1235 profiles processed.
1480 profiles proc